In [1]:
import os

os.environ['CUDA_LAUNCH_BLOCKING'] = "1"

os.environ['WANDB_DISABLED'] = "true"

In [2]:
# !gdown 1qq5gDuEAqWnoviRBDYfbprLFD-MPQyNo
# !gdown 1mq9CWT9MXsH1Y-ihxVL4uEzufQjpBoer
!gdown 1EI12smTmdi4Mlu_lLNULN6p6g9dYEWql

Downloading...
From: https://drive.google.com/uc?id=1EI12smTmdi4Mlu_lLNULN6p6g9dYEWql
To: /kaggle/working/processed_data.zip
100%|██████████████████████████████████████| 6.00M/6.00M [00:00<00:00, 88.0MB/s]


In [3]:
!rm -rf processed_data

!unzip -q processed_data.zip

In [4]:
!pip install -q transformers[torch] accelerate -U

!pip install -q datasets

!pip install -q transformers

!pip install -q iterative-stratification

!pip install -q auto-gptq optimum bitsandbytes

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 336.4/336.4 kB 10.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 66.4 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 98.2 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.5/23.5 MB 75.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 424.1/424.1 kB 23.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.1/69.1 MB 25.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 374.8/374.8 kB 21.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 450.7/450.7 kB 25.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.2/13.2 MB 94.2 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 5.6 MB/s eta 0:00:00

In [51]:
!pip install -q sentence_transformers

/usr/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 7.3 MB/s eta 0:00:00:00:01


In [5]:
import numpy as np

import pandas as pd

import os

from os import path

import glob

from tqdm.notebook import tqdm

from tqdm import tqdm

import string

import copy

from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer,IntervalStrategy,BitsAndBytesConfig

from datasets import Dataset,load_dataset, DatasetDict

# from iterstrat.ml_stratifiers import MultilabelStratifiedKFold

from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay



from sklearn.model_selection import train_test_split,KFold

from sklearn.metrics import classification_report,accuracy_score, f1_score

from sklearn.metrics import precision_recall_fscore_support

from sklearn.utils import shuffle

from sklearn.preprocessing import LabelEncoder

from scipy.special import expit

from peft import prepare_model_for_kbit_training

from peft import LoraConfig, get_peft_model,PeftModel


In [6]:
import torch

import torch.nn as nn

import torch.nn.functional as F

In [7]:


models_list_3=[
        # "FacebookAI/xlm-roberta-large",
      # "google-bert/bert-base-uncased",
        "intfloat/multilingual-e5-large-instruct",

#      "google-bert/bert-base-multilingual-uncased",

          ]



models_name_3=[
# "FacebookAI/xlm-roberta-large",
# "google-bert/bert-base-uncased",
        "intfloat/multilingual-e5-large-instruct",

# "bert",

]


In [376]:
kind="train"

task="track_a"

# langs=["afr","amh","deu","eng","oro","ptbr","rus","som","sum","tir"]
# labels=['Anger','Disgust', 'Fear', 'Joy', 'Sadness', 'Surprise']
# langs=["amh","arq","ary","chn","deu","esp","hau","hin","ibo","kin","mar","orm","pcm","ptbr","ptmz","ron","rus","som","sun","swa","swe","tat","tir","ukr","vmw","yor","ind","jav","xho","zul"]
langs=["mar"]
# langs=["afr"]


# processed_path=f"processed_data/{kinds[0]}/{tasks[0]}/{langs[3]}.csv"



# train_data=pd.read_csv(processed_path)


In [377]:
all_csv=pd.DataFrame()

for lang in langs:

    processed_path=f"processed_data/{kind}/{task}/{lang}.csv"
    # if kind=="dev":
    #    processed_path=processed_path.replace(lang,lang+task[-2:])
    if not os.path.isfile(processed_path):

      print("not found:",processed_path)

      continue
    train_data=pd.read_csv(processed_path)

    train_data.columns = train_data.columns.str.lower()

    all_csv = pd.concat([all_csv, train_data],ignore_index=True)




In [378]:
all_csv

,id,anger,disgust,fear,joy,sadness,surprise,lang,clean_message
0,mar_train_track_a_00001,0,0,0,1,0,0,mar,घरातील झाडांनी माझ्या घराची शोभा वाढवली आहे या...
1,mar_train_track_a_00002,0,0,0,0,0,0,mar,झुंबराची रचना विचित्र होती आणि रंग खराब होता त...
2,mar_train_track_a_00003,1,0,0,0,0,0,mar,उत्सवाच्या ध्वनींचा प्रदूषण सहन करणे अत्यंत कठ...
3,mar_train_track_a_00004,0,0,1,0,0,0,mar,पावसाचा आनंद घेणे चांगले आहे पण मला भीती वाटते...
4,mar_train_track_a_00005,1,0,0,0,0,0,mar,वर्गात सतत निरर्थक बोलणाऱ्या विद्यार्थ्यांची म...
...,...,...,...,...,...,...,...,...,...
2410,mar_train_track_a_02411,0,0,0,0,0,1,mar,पुस्तकाची कथा अशा दिशेने जाईल याची मला कल्पनाच...
2411,mar_train_track_a_02412,0,0,0,1,0,0,mar,जलतरणात विश्वविक्रम मोडल्यामुळे संपूर्ण देशात ...
2412,mar_train_track_a_02413,0,0,0,1,0,0,mar,बक्षीस म्हणून हा अप्रतिम क्रॉकरी सेट मिळाल्यान...
2413,mar_train_track_a_02414,0,0,0,0,0,1,mar,भारतीय आणि बाली संस्कृतीमध्ये अनेक समानता आढळत...


In [379]:
all_csv[all_csv["lang"]=="eng"]

,id,anger,disgust,fear,joy,sadness,surprise,lang,clean_message


In [380]:
# float_columns = all_csv.select_dtypes(include=['float64']).columns
# # Check each column and convert only if there are no NaN values
# for col in float_columns:
#     if all_csv[col].isna().sum() == 0:  # No NaN values in the column
#         print(f"converting {kind} {task} {lang} column {col} to int")
#         all_csv[col] = all_csv[col].astype('int')
#     else:
#         print(f"converting {kind} {task} {lang} column {col} to int")

In [381]:
train_data=all_csv


In [382]:
# train_data = train_data.drop(columns=['disgust'],axis=1)


In [383]:

train_data=train_data.fillna(0)

train_data

,id,anger,disgust,fear,joy,sadness,surprise,lang,clean_message
0,mar_train_track_a_00001,0,0,0,1,0,0,mar,घरातील झाडांनी माझ्या घराची शोभा वाढवली आहे या...
1,mar_train_track_a_00002,0,0,0,0,0,0,mar,झुंबराची रचना विचित्र होती आणि रंग खराब होता त...
2,mar_train_track_a_00003,1,0,0,0,0,0,mar,उत्सवाच्या ध्वनींचा प्रदूषण सहन करणे अत्यंत कठ...
3,mar_train_track_a_00004,0,0,1,0,0,0,mar,पावसाचा आनंद घेणे चांगले आहे पण मला भीती वाटते...
4,mar_train_track_a_00005,1,0,0,0,0,0,mar,वर्गात सतत निरर्थक बोलणाऱ्या विद्यार्थ्यांची म...
...,...,...,...,...,...,...,...,...,...
2410,mar_train_track_a_02411,0,0,0,0,0,1,mar,पुस्तकाची कथा अशा दिशेने जाईल याची मला कल्पनाच...
2411,mar_train_track_a_02412,0,0,0,1,0,0,mar,जलतरणात विश्वविक्रम मोडल्यामुळे संपूर्ण देशात ...
2412,mar_train_track_a_02413,0,0,0,1,0,0,mar,बक्षीस म्हणून हा अप्रतिम क्रॉकरी सेट मिळाल्यान...
2413,mar_train_track_a_02414,0,0,0,0,0,1,mar,भारतीय आणि बाली संस्कृतीमध्ये अनेक समानता आढळत...


In [384]:
float_columns = train_data.select_dtypes(include=['float64']).columns

# Convert those columns to int
train_data[float_columns] = train_data[float_columns].astype('int')
train_data

,id,anger,disgust,fear,joy,sadness,surprise,lang,clean_message
0,mar_train_track_a_00001,0,0,0,1,0,0,mar,घरातील झाडांनी माझ्या घराची शोभा वाढवली आहे या...
1,mar_train_track_a_00002,0,0,0,0,0,0,mar,झुंबराची रचना विचित्र होती आणि रंग खराब होता त...
2,mar_train_track_a_00003,1,0,0,0,0,0,mar,उत्सवाच्या ध्वनींचा प्रदूषण सहन करणे अत्यंत कठ...
3,mar_train_track_a_00004,0,0,1,0,0,0,mar,पावसाचा आनंद घेणे चांगले आहे पण मला भीती वाटते...
4,mar_train_track_a_00005,1,0,0,0,0,0,mar,वर्गात सतत निरर्थक बोलणाऱ्या विद्यार्थ्यांची म...
...,...,...,...,...,...,...,...,...,...
2410,mar_train_track_a_02411,0,0,0,0,0,1,mar,पुस्तकाची कथा अशा दिशेने जाईल याची मला कल्पनाच...
2411,mar_train_track_a_02412,0,0,0,1,0,0,mar,जलतरणात विश्वविक्रम मोडल्यामुळे संपूर्ण देशात ...
2412,mar_train_track_a_02413,0,0,0,1,0,0,mar,बक्षीस म्हणून हा अप्रतिम क्रॉकरी सेट मिळाल्यान...
2413,mar_train_track_a_02414,0,0,0,0,0,1,mar,भारतीय आणि बाली संस्कृतीमध्ये अनेक समानता आढळत...


In [385]:
train_data.dtypes

id               object
anger             int64
disgust           int64
fear              int64
joy               int64
sadness           int64
surprise          int64
lang             object
clean_message    object
dtype: object

In [386]:
train_data = train_data[

train_data['clean_message'].isnull()==False]

train_data.reset_index()

,index,id,anger,disgust,fear,joy,sadness,surprise,lang,clean_message
0,0,mar_train_track_a_00001,0,0,0,1,0,0,mar,घरातील झाडांनी माझ्या घराची शोभा वाढवली आहे या...
1,1,mar_train_track_a_00002,0,0,0,0,0,0,mar,झुंबराची रचना विचित्र होती आणि रंग खराब होता त...
2,2,mar_train_track_a_00003,1,0,0,0,0,0,mar,उत्सवाच्या ध्वनींचा प्रदूषण सहन करणे अत्यंत कठ...
3,3,mar_train_track_a_00004,0,0,1,0,0,0,mar,पावसाचा आनंद घेणे चांगले आहे पण मला भीती वाटते...
4,4,mar_train_track_a_00005,1,0,0,0,0,0,mar,वर्गात सतत निरर्थक बोलणाऱ्या विद्यार्थ्यांची म...
...,...,...,...,...,...,...,...,...,...,...
2410,2410,mar_train_track_a_02411,0,0,0,0,0,1,mar,पुस्तकाची कथा अशा दिशेने जाईल याची मला कल्पनाच...
2411,2411,mar_train_track_a_02412,0,0,0,1,0,0,mar,जलतरणात विश्वविक्रम मोडल्यामुळे संपूर्ण देशात ...
2412,2412,mar_train_track_a_02413,0,0,0,1,0,0,mar,बक्षीस म्हणून हा अप्रतिम क्रॉकरी सेट मिळाल्यान...
2413,2413,mar_train_track_a_02414,0,0,0,0,0,1,mar,भारतीय आणि बाली संस्कृतीमध्ये अनेक समानता आढळत...


In [387]:
train_data

,id,anger,disgust,fear,joy,sadness,surprise,lang,clean_message
0,mar_train_track_a_00001,0,0,0,1,0,0,mar,घरातील झाडांनी माझ्या घराची शोभा वाढवली आहे या...
1,mar_train_track_a_00002,0,0,0,0,0,0,mar,झुंबराची रचना विचित्र होती आणि रंग खराब होता त...
2,mar_train_track_a_00003,1,0,0,0,0,0,mar,उत्सवाच्या ध्वनींचा प्रदूषण सहन करणे अत्यंत कठ...
3,mar_train_track_a_00004,0,0,1,0,0,0,mar,पावसाचा आनंद घेणे चांगले आहे पण मला भीती वाटते...
4,mar_train_track_a_00005,1,0,0,0,0,0,mar,वर्गात सतत निरर्थक बोलणाऱ्या विद्यार्थ्यांची म...
...,...,...,...,...,...,...,...,...,...
2410,mar_train_track_a_02411,0,0,0,0,0,1,mar,पुस्तकाची कथा अशा दिशेने जाईल याची मला कल्पनाच...
2411,mar_train_track_a_02412,0,0,0,1,0,0,mar,जलतरणात विश्वविक्रम मोडल्यामुळे संपूर्ण देशात ...
2412,mar_train_track_a_02413,0,0,0,1,0,0,mar,बक्षीस म्हणून हा अप्रतिम क्रॉकरी सेट मिळाल्यान...
2413,mar_train_track_a_02414,0,0,0,0,0,1,mar,भारतीय आणि बाली संस्कृतीमध्ये अनेक समानता आढळत...


In [388]:
# label_columns =['anger', 'fear', 'joy', 'sadness', 'surprise']
label_columns =['anger', 'disgust', 'fear', 'joy', 'sadness', 'surprise']


In [389]:
train_data['lang'].value_counts()

lang
mar    2415
Name: count, dtype: int64

In [390]:
sum(train_data['lang'].isna())

0

In [391]:
nan_rows = train_data[train_data.isna().any(axis=1)]
nan_rows
# Group by 'lang' and list the rows with NaNs for each language
# nan_by_language = nan_rows.groupby('lang').apply(lambda x: x)

# Display the result
# print(nan_by_language)

,id,anger,disgust,fear,joy,sadness,surprise,lang,clean_message


In [392]:
nan_rows[nan_rows["lang"]=="eng"]

,id,anger,disgust,fear,joy,sadness,surprise,lang,clean_message


In [393]:
train_data[train_data["lang"]=="deu"].dtypes

id               object
anger             int64
disgust           int64
fear              int64
joy               int64
sadness           int64
surprise          int64
lang             object
clean_message    object
dtype: object

In [394]:
nan_rows["lang"].value_counts()

Series([], Name: count, dtype: int64)

In [395]:
nan_rows.dtypes

id               object
anger             int64
disgust           int64
fear              int64
joy               int64
sadness           int64
surprise          int64
lang             object
clean_message    object
dtype: object

In [396]:
# Step 2: Count how many columns are not 0 or 1 for each language
def count_nonzero_nonone_rows(group):
    # Exclude the 'lang' column from counting if needed
    # Use boolean conditions to count values that are not 0 or 1
    return (group.drop(columns='anger') != 0) & (group.drop(columns='anger') != 1).sum(axis=1).sum()

# Group by 'lang' and apply the function to count non-zero, non-one values
non_zero_non_one_counts = train_data.groupby('lang').apply(count_nonzero_nonone_rows)

# Display the results
# result = pd.DataFrame({
#     'row_count': lang_row_counts,
#     'non_zero_non_one_count': non_zero_non_one_counts
# })

print(non_zero_non_one_counts)

              id  disgust   fear    joy  sadness  surprise   lang  \
lang                                                                
mar  0     False    False  False  False    False     False  False   
     1     False    False  False  False    False     False  False   
     2     False    False  False  False    False     False  False   
     3     False    False  False  False    False     False  False   
     4     False    False  False  False    False     False  False   
...          ...      ...    ...    ...      ...       ...    ...   
     2410  False    False  False  False    False     False  False   
     2411  False    False  False  False    False     False  False   
     2412  False    False  False  False    False     False  False   
     2413  False    False  False  False    False     False  False   
     2414  False    False  False  False    False     False  False   

           clean_message  
lang                      
mar  0             False  
     1             Fa

In [397]:
maxi=train_data['clean_message'].apply(len).max()
maxi

175

In [398]:
train_data[train_data['clean_message'].apply(len) == maxi]

,id,anger,disgust,fear,joy,sadness,surprise,lang,clean_message
15,mar_train_track_a_00016,1,0,0,0,1,0,mar,गुरांसाठी चारापाण्याची व्यवस्था नसल्यामुळे आणि...


In [399]:
train_data[train_data['clean_message'].apply(len) == maxi]["clean_message"].values

array(['गुरांसाठी चारापाण्याची व्यवस्था नसल्यामुळे आणि प्राण्यांवर होणाऱ्या अत्याचाराच्या घटनांमध्ये योग्य ती कारवाई होत नसल्याने आम्हा सर्व शेतकऱ्यांमध्ये तीव्र नाराजी आणि निराशा आहे'],
      dtype=object)

In [400]:
# shuffle data
train_data=train_data.sample(frac=1)
# train_data=train_data.sample(100,random_state=42)


In [401]:
# train valid split
train,valid=train_test_split(train_data,test_size=0.2,random_state=42)

In [402]:
x_train, y_train = train['clean_message'].values.tolist(), train[label_columns].values.tolist()

x_valid, y_valid = valid['clean_message'].values.tolist(), valid[label_columns].values.tolist()

In [403]:
y_train_df = pd.DataFrame(y_train, columns=label_columns)

In [404]:
#update label column name

label_distribution = y_train_df.apply(pd.Series.value_counts).T.fillna(0).astype(int)

label_distribution.columns = ['count_0', 'count_1']
label_distribution['sum'] = label_distribution['count_0'] + label_distribution['count_1']
print(label_distribution)

          count_0  count_1   sum
anger        1655      277  1932
disgust      1702      230  1932
fear         1622      310  1932
joy          1572      360  1932
sadness      1587      345  1932
surprise     1684      248  1932


In [405]:
device="cuda" if torch.cuda.is_available() else 'cpu'

In [406]:
from sklearn.utils.class_weight import compute_class_weight

counts_0 = label_distribution['count_0'].to_numpy()
counts_1 = label_distribution['count_1'].to_numpy()

# Compute class-specific weights for each label (each class)
class_weights = []
for i in range(len(counts_0)):
    weight = compute_class_weight('balanced', classes=np.array([0, 1]), y=[0] * counts_0[i] + [1] * counts_1[i])
    class_weights.append(weight)

# Convert the list of weights to a tensor
class_weights_tensor = torch.tensor([w[1] for w in class_weights], dtype=torch.float,device=device)
class_weights_tensor

tensor([3.4874, 4.2000, 3.1161, 2.6833, 2.8000, 3.8952], device='cuda:0')

In [407]:
class Dataset(torch.utils.data.Dataset):

    def __init__(self, encodings, labels):

        self.encodings = encodings

        self.labels = labels



    def __getitem__(self, idx):

        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}

        item['labels'] = torch.tensor(self.labels[idx], dtype=torch.float)

        return item


    def __len__(self):

        return len(self.labels)

In [408]:
!ls output/best-model-v1

ls: cannot access 'output/best-model-v1': No such file or directory


/usr/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()


In [409]:
def build_dataset(x_train,y_train,x_valid,y_valid,MAX_LENGTH):

  train_encodings = tokenizer(x_train, truncation=True, padding="max_length", max_length=MAX_LENGTH)

  valid_encodings = tokenizer(x_valid, truncation=True, padding="max_length", max_length=MAX_LENGTH)

  train_dataset = Dataset(train_encodings, y_train)

  valid_dataset = Dataset(valid_encodings, y_valid)

  return train_dataset,valid_dataset



def train_arg():

    training_args = TrainingArguments(

    output_dir="./output/best-model-v1",

    report_to=None,
    dataloader_pin_memory=False,

    num_train_epochs=3,

    gradient_accumulation_steps=2,

    per_device_train_batch_size=8,

    per_device_eval_batch_size=16,

    learning_rate=2e-5,

    weight_decay=0.01,

    eval_strategy=IntervalStrategy.STEPS,

    save_total_limit=1,

    logging_dir="./logs",

    logging_strategy=IntervalStrategy.STEPS,

    logging_steps=250,

    eval_steps=500,

    load_best_model_at_end=True,

    lr_scheduler_type="cosine_with_restarts",

    warmup_steps=500,

    save_strategy="steps",

    save_steps=500,

  )

    return training_args

In [410]:
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
import numpy as np

def compute_metrics(eval_pred):
    """
    Compute metrics for multi-label classification.

    Args:
        eval_pred: Tuple of (predictions, labels).
            predictions: logits or probabilities of shape (batch_size, num_labels)
            labels: ground truth of shape (batch_size, num_labels)

    Returns:
        metrics: Dictionary containing accuracy, precision, recall, and F1-score.
    """
    # Unpack predictions and labels
    logits, labels = eval_pred

    # Convert logits to binary predictions
    # If your model outputs probabilities (e.g., after sigmoid), use threshold=0.5
    probabilities = expit(logits)
    predictions = (probabilities > 0.5).astype(int)

    # Compute accuracy
    accuracy = accuracy_score(labels, predictions)

    # Compute precision, recall, f1-score
    precision, recall, f1, _ = precision_recall_fscore_support(
        labels, predictions, average=None
    )

    return {
        "accuracy": accuracy,
        "precision": precision,
        "recall": recall,
        "f1": f1
    }

In [411]:
import torch
from torch.utils.data import DataLoader
from tqdm import tqdm
from scipy.special import expit  # For sigmoid activation if needed

def evaluate_model(model, valid_dataset, compute_metrics, batch_size=32, device="cuda",average=None):
    """
    Evaluate a multi-label classification model without using the Hugging Face Trainer.

    Args:
        model: The trained model (Hugging Face model).
        valid_dataset: The validation dataset (torch.utils.data.Dataset).
        compute_metrics: The function to compute evaluation metrics.
        batch_size: Batch size for evaluation.
        device: Device to perform evaluation (e.g., "cuda" or "cpu").

    Returns:
        metrics: Dictionary containing evaluation metrics.
    """
    # Set model to evaluation mode
    model.eval()
    model.to(device)

    # Create DataLoader for the validation dataset
    valid_loader = DataLoader(valid_dataset, batch_size=batch_size, shuffle=False)

    all_predictions = []
    all_labels = []

    # Disable gradient computation for evaluation
    with torch.no_grad():
        for batch in tqdm(valid_loader, desc="Evaluating"):
            # Get inputs and labels
            inputs = {key: val.to(device) for key, val in batch.items() if key != "labels"}
            labels = batch["labels"].to(device)

            # Forward pass: Get logits
            logits = model(**inputs)

            # Apply sigmoid activation to logits if necessary (for multi-label classification)
            probabilities = expit(logits.cpu().numpy())  # Convert logits to probabilities
            predictions = (probabilities > 0.5).astype(int)  # Convert probabilities to binary predictions

            # Collect predictions and labels
            all_predictions.append(predictions)
            all_labels.append(labels.cpu().numpy())
    print()
    # Concatenate all predictions and labels
    all_predictions = np.concatenate(all_predictions, axis=0)
    all_labels = np.concatenate(all_labels, axis=0)

    # Unpack predictions and labels
    logits, labels = all_predictions,all_labels

    # Convert logits to binary predictions
    # If your model outputs probabilities (e.g., after sigmoid), use threshold=0.5
    probabilities = expit(logits)
    predictions = (probabilities > 0.5).astype(int)

    # Compute accuracy
    accuracy = accuracy_score(labels, predictions)

    # Compute precision, recall, f1-score
    precision, recall, f1, _ = precision_recall_fscore_support(
        labels, predictions, average=average
    )
    return {
        "accuracy": accuracy,
        "precision": precision,
        "recall": recall,
        "f1": f1
    },all_predictions

In [412]:
def evaluate_func(trainer):
    metrics = evaluate_model(
    model=model,
    valid_dataset=valid_dataset,
    compute_metrics=compute_metrics,
    batch_size=32,
    device=device
)
    accuracy=metrics["accuracy"]
    precision=metrics["precision"]
    recall=metrics["recall"]
    f1=metrics["f1"]

    return precision, recall, f1, accuracy

In [413]:
MAX_LENGTH = 512

metrics_df = pd.DataFrame(columns=["Models_name", "Precision", "Recall", "F1_score", "Accuracy"])

confusion_matrices = []

In [414]:
quantization_config = BitsAndBytesConfig(

    load_in_4bit=True,

    bnb_4bit_use_double_quant=True,

    bnb_4bit_quant_type="nf4",

    bnb_4bit_compute_dtype=torch.bfloat16,

)

In [415]:
models_list=models_list_3

models_name=models_name_3

x=models_list[0]

In [416]:
token=""

import huggingface_hub

if "gemma" in x or "Phi" in x:

  huggingface_hub.login(token=token)

In [417]:
USE_LORA=False

In [418]:
from transformers import AutoModel
from sentence_transformers import SentenceTransformer

In [419]:


torch.cuda.empty_cache()



print(x)
use_gpu = torch.cuda.is_available()
embedding_model = SentenceTransformer(
        x,
        trust_remote_code=True,
        device="cpu" if not use_gpu else "cuda",
        model_kwargs={
            "torch_dtype": torch.bfloat16 if use_gpu else torch.float32,
            # "attn_implementation": "sdpa"
        },
        # vector_dim must be 12288, 1024, 512, 256
        ## 1024 is recommended
        # set is_text_encoder 'True', if you do not encode image
        # config_kwargs={"is_text_encoder": True, "vector_dim": 1024},
    )
    # We can reduce the max_seq_length from the default of 2048 for faster encoding
embedding_model.max_seq_length = MAX_LENGTH

intfloat/multilingual-e5-large-instruct


In [420]:
def find_target_modules(model):
    # Initialize a Set to Store Unique Layers
    unique_layers = set()

    # Iterate Over All Named Modules in the Model
    for name, module in model.named_modules():
        # Check if the Module Type Contains 'Linear4bit'
        if "Linear4bit" in str(type(module)):
            # Extract the Type of the Layer
            layer_type = name.split('.')[-1]

            # Add the Layer Type to the Set of Unique Layers
            unique_layers.add(layer_type)

    # Return the Set of Unique Layers Converted to a List
    return list(unique_layers)
find_target_modules(embedding_model)

[]

In [421]:
# a=

In [422]:
embedding_model.eval() # embedding_model in training mode (dropout modules are activated)

if USE_LORA:

  # enable gradient check pointing

  embedding_model.gradient_checkpointing_enable()

  target_modules_dict={

      "microsoft/Phi-3.5-mini-instruct":'all-linear',
      "unsloth/Llama-3.2-1B-Instruct":["q_proj","k_proj","v_proj"],
      "unsloth/Qwen2.5-3B-Instruct-bnb-4bit":["q_proj","k_proj","v_proj"],
  }

  target_modules= target_modules_dict.get(x,["q_proj"])



  # enable quantized training

  embedding_model = prepare_model_for_kbit_training(embedding_model)

  config = LoraConfig(

    r=8,

    lora_alpha=32,

    # target_modules=["k_proj"],

    target_modules = target_modules,#TODOOOOOOOOOOOOO

    lora_dropout=0.1,

    bias="none",

    task_type="SEQ_CLS",

  )

  # LoRA trainable version of model

  embedding_model = get_peft_model(embedding_model, config)



  # trainable parameter count

  embedding_model.print_trainable_parameters()

device="cuda" if torch.cuda.is_available() else 'cpu'

In [423]:
device

'cuda'

In [424]:
embedding_model = embedding_model.to(device)

for param in embedding_model.parameters():
    param.data = param.data.contiguous()


In [425]:

# Custom Dataset for batching
class TextDataset(Dataset):
    def __init__(self, texts):
        self.texts = texts

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        # Tokenize a single example
        return self.texts[idx]
        # return self.tokenizer(
        #     self.texts[idx],
        #     truncation=True,
        #     padding="max_length",
        #     max_length=self.max_length,
        #     return_tensors="pt"
        # )


In [426]:

# Batch extraction function
def extract_embeddings(texts, model, batch_size=32):
    dataset = TextDataset(texts)
    dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=False)

    embeddings = []
    model.eval()
    with torch.no_grad():
        for batch in tqdm(dataloader, desc="Extracting embeddings"):
            # Move batch to device
            # input_ids = torch.Tensor(batch['input_ids']).squeeze().to(device)
            # attention_mask = torch.Tensor(batch['attention_mask']).squeeze().to(device)

            # Forward pass through the model
            cls_embeddings = model.encode(batch)
            embeddings.append(cls_embeddings)

    return np.vstack(embeddings)

In [427]:
# Extract embeddings for train and validation datasets
x_train_embeddings = extract_embeddings(x_train, embedding_model, batch_size=32)
x_valid_embeddings = extract_embeddings(x_valid, embedding_model, batch_size=32)

Extracting embeddings:   0%|          | 0/61 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Extracting embeddings:   2%|▏         | 1/61 [00:00<00:12,  4.63it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Extracting embeddings:   3%|▎         | 2/61 [00:00<00:11,  4.99it/s]

...

Extracting embeddings:  88%|████████▊ | 14/16 [00:02<00:00,  5.17it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Extracting embeddings:  94%|█████████▍| 15/16 [00:02<00:00,  5.13it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Extracting embeddings: 100%|██████████| 16/16 [00:03<00:00,  5.32it/s]


In [428]:
# Flatten labels for XGBoost (multi-label classification)
y_train_flat = np.array(y_train)
y_valid_flat = np.array(y_valid)

In [429]:
from sklearn.svm import SVC

In [430]:
svm_models = {}
predictions = {}

for i, label in enumerate(label_columns):
    print(f"Training SVM for label: {label}")

    # Use the computed class weights for balancing the classes
    class_weight = {0: 1, 1: class_weights_tensor[i].item()}  # Adjust the class weights

    svm_model = SVC(class_weight={0: 1, 1: class_weights_tensor[i].item()})

    svm_model.fit(x_train_embeddings, y_train_flat[:, i])
    svm_models[label] = svm_model

    # Predict on validation data
    predictions[label] = svm_model.predict(x_valid_embeddings)

Training SVM for label: anger
Training SVM for label: disgust
Training SVM for label: fear
Training SVM for label: joy
Training SVM for label: sadness
Training SVM for label: surprise


In [431]:
# Evaluate the models
results = []
for i, label in enumerate(label_columns):
    precision, recall, f1, _ = precision_recall_fscore_support(
        y_valid_flat[:, i],
        predictions[label],
        average="binary"
    )
    accuracy = accuracy_score(y_valid_flat[:, i], predictions[label])
    results.append({
        "Label": label,
        "Precision": precision,
        "Recall": recall,
        "F1 Score": f1,
        "Accuracy": accuracy
    })

In [432]:
# Display evaluation results
results_df = pd.DataFrame(results)
print(results_df)

      Label  Precision    Recall  F1 Score  Accuracy
0     anger   0.620253  0.671233  0.644737  0.888199
1   disgust   0.822581  0.739130  0.778626  0.939959
2      fear   0.940299  0.875000  0.906475  0.973085
3       joy   0.794393  0.841584  0.817308  0.921325
4   sadness   0.727273  0.837209  0.778378  0.915114
5  surprise   0.907407  0.777778  0.837607  0.960663


In [433]:
print(f"{x} evaluates:")

for k, res in enumerate(results):

  print(f"Class '{res['Label']}': Precision: {res['Precision']:.4f}, Recall: {res['Recall']:.4f}, F1: {res['F1 Score']:.4f}")

print(f"Overall accuracy: {accuracy:.4f}")

print("-------------------------------------------------------------------------\n")

intfloat/multilingual-e5-large-instruct evaluates:
Class 'anger': Precision: 0.6203, Recall: 0.6712, F1: 0.6447
Class 'disgust': Precision: 0.8226, Recall: 0.7391, F1: 0.7786
Class 'fear': Precision: 0.9403, Recall: 0.8750, F1: 0.9065
Class 'joy': Precision: 0.7944, Recall: 0.8416, F1: 0.8173
Class 'sadness': Precision: 0.7273, Recall: 0.8372, F1: 0.7784
Class 'surprise': Precision: 0.9074, Recall: 0.7778, F1: 0.8376
Overall accuracy: 0.9607
-------------------------------------------------------------------------



In [434]:
# Initialize a DataFrame to store the evaluation results for each language
lang_metrics_df = pd.DataFrame(columns=["Language", "Precision", "Recall", "F1_score", "Accuracy"])

# Iterate over each language
for lang in langs:
    # Filter the validation data for the current language
    lang_valid_data = valid[valid['lang'] == lang]
    lang_x_valid = lang_valid_data['clean_message'].values.tolist()
    lang_y_valid = lang_valid_data[label_columns].values.tolist()

    if len(lang_valid_data) == 0:
        continue

    # Extract embeddings for the current language's validation data
    lang_x_valid_embeddings = extract_embeddings(lang_x_valid, embedding_model)
    print()
    # Initialize metrics storage
    all_precisions, all_recalls, all_f1_scores, all_accuracies = [], [], [], []

    # Evaluate each label separately
    for i, label in enumerate(label_columns):
        # Predict using the corresponding svm model
        lang_predictions = svm_models[label].predict(lang_x_valid_embeddings)

        # Compute metrics
        precision, recall, f1, _ = precision_recall_fscore_support(
            np.array(lang_y_valid)[:, i],  # True labels for this label
            lang_predictions,
            average="binary",
            zero_division=0
        )
        accuracy = accuracy_score(np.array(lang_y_valid)[:, i], lang_predictions)

        # Store metrics
        all_precisions.append(precision)
        all_recalls.append(recall)
        all_f1_scores.append(f1)
        all_accuracies.append(accuracy)

    # Aggregate metrics (average across all labels for the language)
    new_data = pd.DataFrame({
        "Language": [lang],
        "Precision": [np.mean(all_precisions)],
        "Recall": [np.mean(all_recalls)],
        "F1_score": [np.mean(all_f1_scores)],
        "Accuracy": [np.mean(all_accuracies)]
    })

    lang_metrics_df = pd.concat([lang_metrics_df, new_data], ignore_index=True)

# Display the evaluation results for each language
print(lang_metrics_df)


Extracting embeddings:   0%|          | 0/16 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Extracting embeddings:   6%|▋         | 1/16 [00:00<00:03,  4.81it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Extracting embeddings:  12%|█▎        | 2/16 [00:00<00:02,  4.96it/s]

...

Extracting embeddings:  94%|█████████▍| 15/16 [00:02<00:00,  5.11it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Extracting embeddings: 100%|██████████| 16/16 [00:03<00:00,  5.29it/s]



  Language  Precision    Recall  F1_score  Accuracy
0      mar   0.802034  0.790322  0.793855  0.933057


<ipython-input-434-9b6d3eac4dcb>:49: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  lang_metrics_df = pd.concat([lang_metrics_df, new_data], ignore_index=True)


In [435]:
import pickle
# Save the svm models if needed
for label, model in svm_models.items():
    with open(f"svm_model_{label}.pkl", 'wb') as f:
      pickle.dump(model, f)

In [436]:
!zip svm_models_pkl.zip svm_model_*.pkl

/usr/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()


updating: svm_model_anger.pkl (deflated 75%)
updating: svm_model_disgust.pkl (deflated 75%)
updating: svm_model_fear.pkl (deflated 75%)
updating: svm_model_joy.pkl (deflated 75%)
updating: svm_model_sadness.pkl (deflated 75%)
updating: svm_model_surprise.pkl (deflated 75%)
